<a href="https://colab.research.google.com/github/ybressler/intro-to-python/blob/master/Module%204%20%E2%80%93%20Analyzing%20Big%20Data%20%E2%80%93%20Broadway%20Grosses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6 color="#775222">
Module 4 – Analyzing Big Data Grosses for Hamilton on Broadway
</font><br>
*We'll be using python to analyze big data from the Broadway musical Hamilton.*


---

**Outline:**
* load data
* rename columns
* clean data
* aggregate data (make groups)
* use time series aspect
* visualize

In [0]:
import re
import json
import requests
import pandas as pd
import datetime
import matplotlib.pyplot as plt

# 1. Load & Preprocess data

Broadway Grosses for Hamilton
* Broadway World: https://www.broadwayworld.com/grosses/HAMILTON
* download all data to excel – then convert to csv – then upload

**Alternatively:**
*  use the following file 
* _(downloaded on 04/06/2020)_
* https://storage.googleapis.com/yb-intro-to-python/hamilton_data_04-06-2020.csv

## 1.0 Actually load the data

In [0]:
# Load data for Hamilton grosses
data_path = "https://storage.googleapis.com/yb-intro-to-python/hamilton_data_04-06-2020.csv"
data = requests.get(data_path).text # data is text

In [0]:
# Investigate data
print(data)
print(type(data))

In [0]:
# load directly into a dataframe
df = pd.read_csv(data_path)

# View only the first 5 rows
df.head(5)

Data isn't right... We need to rename some columns, clean up some of the rows and properly structure some of the data

In [0]:
# save a backup so we can work off of it
df_backup = df.copy(deep=True)

## 1.1 Rename Columns

In [0]:
# Rename them one at a time
df.rename(columns = {"This Week's Gross":"Gross"})

# It won't get saved unless you tell it to!
# If the column doesn't exist, no problem!
df.rename(columns = {"This Week's Gross":"Gross"}, inplace=True)

# Alternatively, make a copy of the df (a bad idea)
# df = df.rename(columns = {"This Week's Gross":"Gross"})

df.head(5)

**An easier way to rename many columns at one is to create a dictionary, then pass that to the rename function:**

In [0]:
# a dict of new column names
new_cols = {
    "This Week's Gross":"Gross",
    "Last Week's Gross":"Gross Last Week",
    "Diff $":"Gross Diff",
    "Potential Gross":"Gross Potential Value",
    "Gross % of Potential":"Gross Potential",
    "Average Ticket":"Average Ticket Price",
    "Top Ticket":"Top Ticket Price",
    "Seats":"Seats Sold",
    "Total":"Total Seats",
    "Per":"N Performances",
    "This Week %":"Capacity",
    "Last Week %":"Capacity Last Week",
    "Diff %":"Capacity Diff"
}

# rename your columns
df.rename(columns = new_cols, inplace=True)

**Alternatively, you can use lists:** *(the order matters though!)*

In [0]:
cols = ['Week', 'Gross', 'Gross Last Week', 'Gross Diff',
       'Gross Potential Value', 'Gross Potential', 'Average Ticket Price',
       'Top Ticket Price', 'Seats Sold', 'Total Seats', 'N Performances',
       'Capacity', 'Capacity Last Week', 'Capacity Diff']

df.columns = cols

df.head(3)

**You can also perform series like operations**

In [0]:
# make them all lowercase
cols = df.columns.str.lower()
print(cols)

# replace
cols = df.columns.str.replace(" ", "_")
print(cols)

# replace using regex – \s+ gets rid of all empty spaces
cols = df.columns.str.replace("\s+", "_", regex=True)
print(cols)

## 1.2 Drop rows & columns



Verify that your columns match this list
```
['Week', 'Gross', 'Gross Last Week', 'Gross Diff',
'Gross Potential Value', 'Gross Potential', 'Average Ticket Price',
'Top Ticket Price', 'Seats Sold', 'Total Seats', 'N Performances',
'Capacity', 'Capacity Last Week', 'Capacity Diff']
```



**Drop a row:**

In [0]:
df.head(3)

In [0]:
# Drop the first row
df.drop(0, inplace=True, errors="ignore")

df.head(3)

**Drop a column:**

In [0]:
# create a list of columns to drop --> then drop them!
drop_cols = ["Gross Last Week","Gross Diff", "Capacity Last Week", "Capacity Diff"]
df.drop(columns=drop_cols, inplace=True, errors="ignore")

df.head(3)

## 1.3 Change data type – datetime

Almost all of our data types are not correct. Let's fix this.

In [0]:
# Check your current data types
df.dtypes

```
# Recall the methods to convert from string to datetime

# DATETIME METHOD
dt = '04/16/2017'
dt = datetime.datetime.strptime(dt, "%m/%d/%Y")
print(dt)

# PANDAS METHOD
dt = '04/16/2017'
dt = pd.to_datetime(dt)
print(dt)
```

In [0]:
# convert to datetime
df["Week"] = df["Week"].apply(pd.to_datetime)

In [0]:
# Check your current data types
df.dtypes

## 1.5 Change data type – numeric:

In [0]:
# Manually replace "$" and "," --> then convert to int
df["Gross"] = df["Gross"].astype(str).str.replace("$","").str.replace(",","").astype(int)

# A neater way –  Use regex!
# df["Gross"].str.replace("[\$,]","", regex=True)

Create a function to do this for you:

In [0]:
def convert_str_to_num(s):
  """
  This converts a messy string to an integer or float
  
  params:
    s: a string
  
  output:
    an int or float
  """

  # if it's already an int or a float, don't do anything...
  if type(s) in [int, float]:
    return s

  # otherwise
  s = re.sub("[\$,%]", "", s)

  # if empty
  if len(s)<1:
    return None
  # otherwise
  if "." in s:
    return float(s)
  else:
    return int(s)

df["Top Ticket Price"].apply(convert_str_to_num)

**Use this function on many columns:**

In [0]:
num_cols = [
            "Gross",
            "Gross Potential Value",
            'Gross Potential',
            'Average Ticket Price',
            'Top Ticket Price', 
            'Seats Sold', 
            'Total Seats',
            'N Performances',
            'Capacity',
            'Pay Attention!'
          ]


In [0]:
# Let's verify that these columns are in our df:

for col in num_cols:
  if col not in df.columns:
    print(f"`{col}` not in columns!")

In [0]:
# A neat way to do this is:

new_num_cols = []
for col in num_cols:
  if col in df.columns:
    new_num_cols.append(col)

# finally
new_num_cols

In [0]:
# Even neater – called a list operator
new_num_cols = [col for col in num_cols if col in df.columns]

new_num_cols

In [0]:
# Now, let's go ahead and change the datatypes
for col in new_num_cols:
  df[col] = df[col].apply(convert_str_to_num)


In [0]:
df.dtypes

## 1.4 Create a column for year

In [0]:
# Create a new column
df["Year"] = df["Week"].dt.year

df.head(3)

# 2. Load data – with all our preprocessing code upfront

In [0]:
# First, load your functions...
def convert_str_to_num(s):
  """
  This converts a messy string to an integer or float
  
  params:
    s: a string
  
  output:
    an int or float
  """

  # if it's already an int or a float, don't do anything...
  if type(s) in [int, float]:
    return s

  # otherwise
  s = re.sub("[\$,%]", "", s)

  # if empty
  if len(s)<1:
    return None
  # otherwise
  if "." in s:
    return float(s)
  else:
    return int(s)


In [0]:
data_path = "https://storage.googleapis.com/yb-intro-to-python/hamilton_data_04-06-2020.csv"

# load directly into a dataframe
df = pd.read_csv(data_path)

# a dict of new column names
new_cols = {
    "This Week's Gross":"Gross",
    "Last Week's Gross":"Gross Last Week",
    "Diff $":"Gross Diff",
    "Potential Gross":"Gross Potential Value",
    "Gross % of Potential":"Gross Potential",
    "Average Ticket":"Average Ticket Price",
    "Top Ticket":"Top Ticket Price",
    "Seats":"Seats Sold",
    "Total":"Total Seats",
    "Per":"N Performances",
    "This Week %":"Capacity",
    "Last Week %":"Capacity Last Week",
    "Diff %":"Capacity Diff"
}

# rename your columns
df.rename(columns = new_cols, inplace=True)

# Drop the first row
df.drop(0, inplace=True, errors="ignore")

# convert to datetime
df["Week"] = df["Week"].apply(pd.to_datetime)


num_cols = [
            "Gross",
            "Gross Potential Value",
            'Gross Potential',
            'Average Ticket Price',
            'Top Ticket Price', 
            'Seats Sold', 
            'Total Seats',
            'N Performances',
            'Capacity',
            'Pay Attention!'
          ]

# Filter with a list operator
num_cols = [col for col in num_cols if col in df.columns]

# Now, let's go ahead and change the datatypes
for col in new_num_cols:
  df[col] = df[col].apply(convert_str_to_num)


# Create a new column
df["Year"] = df["Week"].dt.year

df.head(3)

## 2.1 See the power of pandas in action:

In [0]:
df.info()

In [0]:
df.describe()

In [0]:
df[["Gross", "Gross Potential Value"]].plot(title="Gross & Potential")

# Set to a log axis
# plt.yscale("log")

plt.show()

In [0]:
df[["Average Ticket Price","Top Ticket Price"]].plot(title="Ticket Prices")

plt.show()

# 3. Analyze

Let's answer some simple Q's

## 3.1 What was the best week per year for `Gross`?

In [0]:
# Here's how you get the max value
df.groupby("Year")["Gross"].max()

In [0]:
# Here's how you get the date of the max value
df.set_index("Week").groupby("Year")["Gross"].idxmax()

In [0]:
# Put those together to get:
df_agg = df.set_index("Week").groupby("Year")["Gross"].agg(["idxmax", max])

df_agg

In [0]:
# Rename your columns:
df_agg.columns = ["Week", "Gross"]
df_agg

In [0]:
# Try plotting!
df_agg["Gross"].plot(kind="bar")

## 3.2 What was the best performing week for Hamilton ever?

In [0]:
df["Gross"].max()

In [0]:
# Get the index
max_idx = df["Gross"].idxmax()

# Get the specific row
max_row = df.iloc[max_idx]

max_dt = max_row["Week"]
max_gross = max_row["Gross"]

print(f"The best performing week for Hamilton *ever* occured on {max_dt} with a gross of ${max_gross:,}")

## 3.3 How many people saw the show since Hamilton won their Tony award?

`date of Hamilton's win was June 12, 2016`

In [0]:
dt_tony = pd.to_datetime("June 12, 2016")
dt_tony

In [0]:
# Filter using the following
df[df["Week"]>= dt_tony]

In [0]:
# Now get the sum
n = df[df["Week"]>= dt_tony]["Seats Sold"].sum()

print(f"{n:,} people saw Hamilton since they won their Tony award ({dt_tony})")

## 3.4 How many people saw Hamilton each year?

In [0]:
df.groupby("Year")["Seats Sold"].sum().plot(kind="bar")

## 3.5 Visualize how average ticket price changed over time. Distinguish before and after Hamilton won the Tony award.

In [0]:
# this is what this looks like without any changes
df["Average Ticket Price"].plot()

In [0]:
# reuse the code for dt
dt_tony = pd.to_datetime("June 12, 2016")

# partition color based on before or after the tony award
df["color"] = df["Week"].apply(lambda x: "green" if x>=dt_tony else "blue")

In [0]:
plt.scatter(x="Week", y="Average Ticket Price", color ="color", data=df)

# add a vertical line
plt.vlines(dt_tony, 0,df["Average Ticket Price"].max(), color="red", alpha=0.5, linestyle="--")
plt.show()

All of this is much easier if you convert to a time series dataframe

In [0]:
df_ts = df.set_index("Week", drop=True)

# Groupby to use a diff color on the same line
for label, group_df in df_ts.groupby("color"):
  plt.plot(group_df["Average Ticket Price"], color=label)

# add a vertical line
plt.vlines(dt_tony, 0,df["Average Ticket Price"].max(), color="red", alpha=0.5, linestyle="--", label ="70th Tony Awards")

# add another
dt_tony_2 = pd.to_datetime("June 11, 2017")
plt.vlines(dt_tony_2, 0,df["Average Ticket Price"].max(), color="red", alpha=0.5, linestyle="--", label ="71st Tony Awards")

# display legend
plt.legend()

# ====================================

# Give it a title:
plt.title("Average Ticket Price for Hamilton over Time", weight="bold")

# Y-label
plt.ylabel("Avg. Ticket Price ($)", weight="bold")


# ====================================

plt.show()

# 3.6 Which month has the poorest attendance? 

In [0]:
# create a new column for month:
df["Month"] = df["Week"].dt.month

# Use capacity since N Performance / Month may vary
df_agg = df.groupby("Month")["Capacity"].mean()

n = df_agg.idxmin()
print(f"The poorest month is {n}")

**Visualize the results!**

In [0]:
df_agg.plot(
    kind="bar",
    # "ac7732" is hamilton's color!
    color="#ac7732",
    figsize=(12,6)
    )

plt.title("% Capacity per month\nHamilton on Broadway",color="#775222", weight="bold", size=14)
plt.ylabel("% Capacity", color="#775222", weight="bold")

# Doesn't look that different... Try shifting to log scale...
plt.yscale("log")


# To save
# plt.savefig("hamilton_monthly_attendance.png",dpi=150)

plt.show()

# 4. Conclusion

By now, you're able to explore data on your own and answer whatever interesting Q's you might have. I'd encourage you to practice on this data set (since you're already familiar with it) and others!

## 4.1 A question you want to solve about the dataset ?

In [0]:
# Code here to solve the question